<a href="https://colab.research.google.com/github/trollmannen/Data-Science/blob/main/WoW-Auction-House-EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [14]:
CLIENT_ID = ""
CLIENT_SECRET = ""
REGION = "eu" #"us"
LOCALE = "en_GB" #"en_US"

In [3]:
#!pip install python-blizzardapi

In [4]:
from blizzardapi import BlizzardApi
api_client = BlizzardApi(CLIENT_ID, CLIENT_SECRET)

## Part 1: Gather & Store Data

In [5]:
import requests
def create_access_token():
  data = { 'grant_type': 'client_credentials' }
  response = requests.post(f'https://{REGION}.battle.net/oauth/token',
                           data=data, auth=(CLIENT_ID, CLIENT_SECRET))
  return response.json()['access_token']

access_token = create_access_token()

In [6]:
def get_connected_realms_index(access_token):
  connected_realms_index = {}
  response = requests.get(f'https://{REGION}.api.blizzard.com/data/wow/search/'
                          f'connected-realm?namespace=dynamic-{REGION}'
                          f'&access_token={access_token}')
  for connected_realm in response.json()['results']:
    for realm in connected_realm['data']['realms']:
      connected_realms_index[realm['slug']] = connected_realm['data']['id']

  return pd.Series(connected_realms_index)

connected_realms_index = get_connected_realms_index(access_token)

In [7]:
def get_auctions(connected_realm_id):
  auction_id,item_id,unit_price,buyout,bid,quantity = ([] for i in range(6))
  response = api_client.wow.game_data.get_auctions(REGION,
                                                   LOCALE,
                                                   connected_realm_id)
  for auction in response['auctions']:
    auction_id.append(auction['id'])
    item_id.append(auction['item']['id'])

    if 'unit_price' in auction:
      unit_price.append(auction['unit_price'])
    else:
      unit_price.append(pd.NA)

    if 'buyout' in auction:
      buyout.append(auction['buyout'])
    else:
      buyout.append(pd.NA)

    if 'bid' in auction:
      bid.append(auction['bid'])
    else:
      bid.append(pd.NA)

    quantity.append(auction['quantity'])

  auctions = pd.DataFrame(list(zip(auction_id,item_id,
                                   unit_price,buyout,bid,quantity)),
                          columns=['auction_id','item_id',
                                   'unit_price','buyout','bid','quantity'])
  return auctions

In [8]:
all_auctions = pd.DataFrame(columns=['auction_id','item_id',
                                     'unit_price','buyout','bid','quantity'])

In [9]:
from tqdm import tqdm
for connected_realm_id in tqdm(connected_realms_index.unique()):
    all_auctions = pd.concat([get_auctions(connected_realm_id),
                              all_auctions], axis=0)


100%|██████████| 92/92 [02:47<00:00,  1.82s/it]


In [10]:
def get_item_name(item_id):
  item = api_client.wow.game_data.get_item(region=REGION,
                                           locale=LOCALE,
                                           item_id=item_id)
  return item['name']

#all_auctions['item'] = np.vectorize(get_item_name)(all_auctions['item_id'])

In [11]:
all_auctions.to_csv('all_auctions.csv',index=False)

## Part 2: Clean & Organize Data

In [12]:
all_auctions = pd.read_csv('all_auctions.csv',index_col='auction_id')

In [13]:
all_auctions

,item_id,unit_price,buyout,bid,quantity
auction_id,,,,,
1335559057,14047,5200.0,NaN,NaN,57
1335961382,37921,144200.0,NaN,NaN,14
1335861176,43465,26400.0,NaN,NaN,1
1335784242,162517,668100.0,NaN,NaN,9
1335476437,171830,150000.0,NaN,NaN,800
...,...,...,...,...,...
1336847893,170554,282000.0,NaN,NaN,200
1336847898,168589,305000.0,NaN,NaN,119
1336847901,168586,305000.0,NaN,NaN,179
